## Importamos las librerias necesarias para el proyecto

In [59]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

### Cargamos las imagenes para generear los conjuntos de entrenamiento

In [60]:
# Definir las rutas a las carpetas de imágenes
train_data_dir = r'C:/Users/tomif/Desktop/AI Detectora de Cancer de Piel/train'  


In [61]:
# Crear un generador de imágenes con aumentación
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Escala los píxeles entre 0 y 1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Dividimos el dataset en 80% entrenamiento y 20% validación
)

In [62]:
# Generador para los datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),  # Tamaño al que se redimensionan las imágenes
    batch_size=32,
    class_mode='binary',  # Como tenemos dos clases, usamos 'binary'
    subset='training'
)

Found 1246 images belonging to 2 classes.


In [63]:
# Generador para los datos de validación
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 311 images belonging to 2 classes.


## Creacion del Modelo

In [64]:
# Construimos el modelo
model = Sequential()

In [65]:
# Primera capa de convolución y pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [66]:
# Segunda capa de convolución y pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [67]:
# Tercera capa de convolución y pooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [68]:
# Aplanamiento
model.add(Flatten())

In [69]:
# Capa totalmente conectada (densa)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Regularización para evitar sobreajuste

In [70]:
# Capa de salida
model.add(Dense(1, activation='sigmoid'))  # 'sigmoid' para clasificación binaria

In [71]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
# Resumen del modelo
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

## Entrenamiento del Modelo

In [73]:
# Definir el peso de las clases en función del desbalance
total_images = 288 + 958
weight_for_benign = total_images / (2 * 288)
weight_for_malignant = total_images / (2 * 958)

class_weight = {0: weight_for_benign, 1: weight_for_malignant}

# Entrenar el modelo con los pesos ajustados
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    class_weight=class_weight,  # Pasar el ajuste de pesos de clase
    epochs=10
)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 245ms/step - accuracy: 0.5691 - loss: 0.7753 - val_accuracy: 0.7708 - val_loss: 0.5707
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5938 - loss: 0.9087 - val_accuracy: 0.8261 - val_loss: 0.6537
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 240ms/step - accuracy: 0.5288 - loss: 0.6533 - val_accuracy: 0.7778 - val_loss: 0.5751
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6875 - loss: 0.7465 - val_accuracy: 0.7391 - val_loss: 0.5824
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 242ms/step - accuracy: 0.7818 - loss: 0.5707 - val_accuracy: 0.2639 - val_loss: 0.7754
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3750 - loss: 0.6387 - val_accuracy: 0.2174 - val_loss: 0.7596
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 241ms/step - accuracy: 0.6693 - loss: 0.5728 - val_accuracy: 0.7882 - val_loss: 0.6055
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9062 - loss: 0.4732 - val_accuracy: 0.8696

## Evaluacion del modelo y guardado del mismo 

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(validation_generator)
print(f'Precisión en el set de validación: {accuracy * 100:.2f}%')

# Guardar el modelo entrenado
model.save('modelo_cancer_piel.h5')

In [75]:
print(f"Imágenes benignas: {train_generator.classes.tolist().count(0)}")
print(f"Imágenes malignas: {train_generator.classes.tolist().count(1)}")

Imágenes benignas: 288
Imágenes malignas: 958


In [76]:
# Revisar el mapeo de clases
print(train_generator.class_indices)

{'Benigno': 0, 'Maligno': 1}
